<a href="https://colab.research.google.com/github/DamnCrab/Real-ESRGAN-Inference-Demo/blob/main/Real_ESRGAN_Inference_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Real-ESRGAN Inference Demo

[![arXiv](https://img.shields.io/badge/arXiv-Paper-<COLOR>.svg)](https://arxiv.org/abs/2107.10833)
[![GitHub Stars](https://img.shields.io/github/stars/xinntao/Real-ESRGAN?style=social)](https://github.com/xinntao/Real-ESRGAN)
[![download](https://img.shields.io/github/downloads/xinntao/Real-ESRGAN/total.svg)](https://github.com/xinntao/Real-ESRGAN/releases)

This is a **Practical Image Restoration Demo** of our paper [''Real-ESRGAN: Training Real-World Blind Super-Resolution with Pure Synthetic Data''](https://arxiv.org/abs/2107.10833).
We extend the powerful ESRGAN to a practical restoration application (namely, Real-ESRGAN), which is trained with pure synthetic data. <br>
The following figure shows some real-life examples.

<img src="https://raw.githubusercontent.com/xinntao/Real-ESRGAN/master/assets/teaser.jpg" width="100%">

We provide a pretrained model (*RealESRGAN_x4plus.pth*) with upsampling X4.<br>
**Note that RealESRGAN may still fail in some cases as the real-world degradations are really too complex.**<br>
Moreover, it **may not** perform well on **human faces, text**, *etc*, which will be optimized later.
<br>

You can also find a **Portable Windows/Linux/MacOS executable files for Intel/AMD/Nvidia GPU.** in our [GitHub repo](https://github.com/xinntao/Real-ESRGAN). <br>
This executable file is **portable** and includes all the binaries and models required. No CUDA or PyTorch environment is needed.<br>
This executable file is based on the wonderful [Tencent/ncnn](https://github.com/Tencent/ncnn) and [realsr-ncnn-vulkan](https://github.com/nihui/realsr-ncnn-vulkan) by [nihui](https://github.com/nihui).

# 1. Preparations
Before start, make sure that you choose
* Runtime Type = Python 3
* Hardware Accelerator = GPU

in the **Runtime** menu -> **Change runtime type**

Then, we clone the repository, set up the envrironment, and download the pre-trained model.

In [1]:
#检测使用的显卡
! /opt/bin/nvidia-smi
! nvcc --version
#注册https://wandb.ai/authorize 填api 获得实时监控网址
#!pip install wandb
#import wandb
#wandb.init()

Wed Aug  4 04:38:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!sudo apt install build-essential curl cmake pkg-config
!git clone https://github.com/DamnCrab/ffmpeg-build-script.git 
%cd ffmpeg-build-script 
! ./build-ffmpeg --enable-nonfree --build 

##原项目 https://github.com/markus-perl/ffmpeg-build-script 请fork后删除build-ffmpeg内webp安装 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.1).
curl is already the newest version (7.58.0-2ubuntu3.13).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
Cloning into 'ffmpeg-build-script'...
remote: Enumerating objects: 946, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 946 (delta 46), reused 50 (delta 26), pack-reused 867
Receiving objects: 100% (946/946), 304.79 KiB | 1.64 MiB/s, done.
Resolving deltas: 100% (570/570), done.
/content/ffmpeg-build-script/ffmpeg-build-script
ffmpeg-build-script v1.28

Using 2 make jobs simultaneously.

building pkg-config
... Done
Extracted pkg-config-0.29.2.tar.gz
$ ./configure --silent --prefix=/content/ffmpeg-build-script/ffmpeg-build-script/workspace --with-pc-path=/content/ffmpeg-build-script/ff

In [5]:
%cd /content/ffmpeg-build-script
!rm -rf ffmpeg-build-script

/content/ffmpeg-build-script


In [ ]:
pkg-config lv2

In [14]:

%cd nv-codec-headers
!ls



[Errno 2] No such file or directory: 'nv-codec-headers'
/content/nv-codec-headers
ffnvcodec.pc  ffnvcodec.pc.in  include	Makefile  README


In [7]:


!cd nv-codec-headers && sudo make install && cd –


sed 's#@@PREFIX@@#/usr/local#' ffnvcodec.pc.in > ffnvcodec.pc
install -m 0755 -d '/usr/local/include/ffnvcodec'
install -m 0644 include/ffnvcodec/*.h '/usr/local/include/ffnvcodec'
install -m 0755 -d '/usr/local/lib/pkgconfig'
install -m 0644 ffnvcodec.pc '/usr/local/lib/pkgconfig'
/bin/bash: line 0: cd: –: No such file or directory


In [15]:
#git clone https://git.videolan.org/git/ffmpeg/nv-codec-headers.git
#%cd nv-codec-headers


!make
!sudo make install

sed 's#@@PREFIX@@#/usr/local#' ffnvcodec.pc.in > ffnvcodec.pc
sed 's#@@PREFIX@@#/usr/local#' ffnvcodec.pc.in > ffnvcodec.pc
install -m 0755 -d '/usr/local/include/ffnvcodec'
install -m 0644 include/ffnvcodec/*.h '/usr/local/include/ffnvcodec'
install -m 0755 -d '/usr/local/lib/pkgconfig'
install -m 0644 ffnvcodec.pc '/usr/local/lib/pkgconfig'


In [21]:
!ls

nv-codec-headers  sample_data


In [25]:
!git clone https://git.ffmpeg.org/ffmpeg.git ffmpeg

Cloning into 'ffmpeg'...
remote: Enumerating objects: 4406, done.
remote: Counting objects: 100% (4406/4406), done.
remote: Compressing objects: 100% (3590/3590), done.
remote: Total 630890 (delta 2848), reused 1009 (delta 812)
Receiving objects: 100% (630890/630890), 153.48 MiB | 14.08 MiB/s, done.
Resolving deltas: 100% (506741/506741), done.


In [26]:
!sudo apt install nasm yasm

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  nasm yasm
0 upgraded, 2 newly installed, 0 to remove and 40 not upgraded.
Need to get 765 kB of archives.
After this operation, 4,995 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 nasm amd64 2.13.02-0.1 [359 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 yasm amd64 1.3.0-2build1 [406 kB]
Fetched 765 kB in 1s (604 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting p

In [ ]:
!sudo apt install libx264-dev libx265-dev libnuma-dev libvpx-dev libfdk-aac-dev libmp3lame-dev libopus-dev
# Ubuntu 20.04已支持自动安装libaom，不需要手动下载源码，非常方便
#!sudo apt install libaom-dev

#!sudo apt install -y libgsm-devel libmp3lame-devel libgnutlsxx-devel ladspa-devel libass-devel libbluray-devel libcelt-devl libdc1394-devel openjpeg-devel openjpeg2-devel libopus-devel speex-devel  libtwolame-devel libvidstab-devel libvorbis-devel libvpx-devel libwebp-devel libx265-devel libxvidcore-devel zimg-devel zvbi-devel libcdio-devel cdparanoia-devel libcdio-paranoia-devel libva-devel libvdpau-devel

In [30]:
!git clone https://git.videolan.org/git/ffmpeg/nv-codec-headers.git
%cd nv-codec-headers  
!sudo make install  
%cd ..

## 通过pkg-config 验证ffnvcodec，如果找不到，把ffnvcodec.pc路径添加到PKG_CONFIG_PATH
!pkg-config --modversion ffnvcodec

fatal: destination path 'nv-codec-headers' already exists and is not an empty directory.
/content/nv-codec-headers
sed 's#@@PREFIX@@#/usr/local#' ffnvcodec.pc.in > ffnvcodec.pc
install -m 0755 -d '/usr/local/include/ffnvcodec'
install -m 0644 include/ffnvcodec/*.h '/usr/local/include/ffnvcodec'
install -m 0755 -d '/usr/local/lib/pkgconfig'
install -m 0644 ffnvcodec.pc '/usr/local/lib/pkgconfig'
/content
11.1.5.0


In [31]:
%cd ffmpeg

/content/ffmpeg


In [ ]:
!./configure --prefix=${workspace}/install/x86 --enable-cuda --enable-cuvid --enable-nvenc --enable-nonfree --disable-lzma --disable-zlib --disable-sdl2  --disable-ffplay --disable-avdevice --enable-shared --extra-cflags=" -fPIC "

In [33]:
!make
!make install

GEN	libavutil/libavutil.version
GEN	libswscale/libswscale.version
GEN	libswresample/libswresample.version
GEN	libavcodec/libavcodec.version
GEN	libavformat/libavformat.version
GEN	libavfilter/libavfilter.version
CC	libavfilter/aeval.o
CC	libavfilter/af_acontrast.o
CC	libavfilter/af_acopy.o
CC	libavfilter/af_acrossover.o
CC	libavfilter/af_acrusher.o
CC	libavfilter/af_adeclick.o
CC	libavfilter/af_adelay.o
CC	libavfilter/af_adenorm.o
CC	libavfilter/af_aderivative.o
CC	libavfilter/af_aecho.o
CC	libavfilter/af_aemphasis.o
CC	libavfilter/af_aexciter.o
CC	libavfilter/af_afade.o
CC	libavfilter/af_afftdn.o
CC	libavfilter/af_afftfilt.o
CC	libavfilter/af_afir.o
CC	libavfilter/af_aformat.o
CC	libavfilter/af_afreqshift.o
CC	libavfilter/af_afwtdn.o
CC	libavfilter/af_agate.o
CC	libavfilter/af_aiir.o
CC	libavfilter/af_alimiter.o
CC	libavfilter/af_amerge.o
CC	libavfilter/af_amix.o
CC	libavfilter/af_amultiply.o
CC	libavfilter/af_anequalizer.o
CC	libavfilter/af_anlmdn.o
CC	libavfilter/af_anlms.o
CC	libav

In [40]:
!ffmpeg -version

ffmpeg version 4.3.2-0york0~18.04 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
configuration: --prefix=/usr --extra-version='0york0~18.04' --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --e

In [38]:
%cd nv-codec-headers
!git tag

/content/nv-codec-headers
n10.0.26.0
n10.0.26.1
n10.0.26.2
n11.0.10.0
n11.0.10.1
n8.0.14.1
n8.0.14.2
n8.0.14.4
n8.0.14.5
n8.0.14.6
n8.0.14.7
n8.0.14.8
n8.0.14.9
n8.1.24.0
n8.1.24.1
n8.1.24.10
n8.1.24.11
n8.1.24.12
n8.1.24.13
n8.1.24.2
n8.1.24.4
n8.1.24.5
n8.1.24.6
n8.1.24.7
n8.1.24.8
n8.1.24.9
n8.2.15.1
n8.2.15.10
n8.2.15.11
n8.2.15.12
n8.2.15.2
n8.2.15.3
n8.2.15.4
n8.2.15.5
n8.2.15.6
n8.2.15.7
n8.2.15.8
n8.2.15.9
n9.0.18.0
n9.0.18.1
n9.0.18.2
n9.0.18.3
n9.0.18.4
n9.0.18.5
n9.1.23.0
n9.1.23.1
n9.1.23.2
n9.1.23.3


In [ ]:
# Clone Real-ESRGAN and enter the Real-ESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr
# !pip install -r requirements.txt
# Download the pre-trained model
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models

# 2. Upload Images

Upload the images to be processed by Real-ESRGAN

In [ ]:
#网页上传方式
import os
from google.colab import files
import shutil

upload_folder = 'upload'
result_folder = 'results'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.mkdir(upload_folder)
os.mkdir(result_folder)

# upload images
uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)



Saving 序列 02_1.mp4 to 序列 02_1.mp4
move 序列 02_1.mp4 to upload/序列 02_1.mp4


In [ ]:
#Drive上传方式 需要授权
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWgoeRg0inGmB7gJvbAdXg9k-Dmc88F_HzCUJq3dHLQadL8c3vV0Brc
Mounted at /content/drive


In [ ]:
import os
from google.colab import files
import shutil
upload_folder = 'upload'
result_folder = 'results'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.mkdir(upload_folder)
os.mkdir(result_folder)

filename='1.mkv' #填文件名
dst_path1 = os.path.join("/content/drive/MyDrive/", filename)

shutil.copy(dst_path1,"/content/Real-ESRGAN/upload/")




'/content/Real-ESRGAN/upload/1.mkv'

In [ ]:
# 检测输入文件FPS。
tmp_path = os.path.join("/content/Real-ESRGAN/upload/", filename)
import cv2

video = cv2.VideoCapture(tmp_path)

fps = video.get(cv2.CAP_PROP_FPS)
print("帧率: {0}".format(fps))
frame_num = video.get(cv2.CAP_PROP_FRAME_COUNT)
print("帧数: {0}".format(fps))

#av1编码有bug
#print(f'fps= {fps}')


Frames per second using video.get(cv2.CAP_PROP_FPS) : 60.0
fps= 60.0


In [39]:
#更新ffmpeg 
!add-apt-repository ppa:jonathonf/ffmpeg-4 -y
!apt-get update
!apt-get install ffmpeg

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [62.9 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:13 https://developer.download.nvi

In [ ]:
shell ffmpeg -version

ffmpeg version 4.3.2-0york0~18.04 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
configuration: --prefix=/usr --extra-version='0york0~18.04' --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --e

In [ ]:
%shell ffmpeg -i '/content/Real-ESRGAN/upload/{filename}' -codec copy -map 0 -f segment -segment_list out.csv -segment_frames 1000,2000,3000,4000,5000 out%03d.mkv

SyntaxError: ignored

In [ ]:
!apt-get install nodejs-dev node-gyp libssl1.0-dev
!apt-get install npm
!$ npm install --global ffmpeg-progressbar-cli


In [ ]:
! npm install --global ffmpeg-progressbar-cli

In [ ]:
#拆帧
#filename='1.mkv'
%shell rm -rf '/content/Real-ESRGAN/frame'
%shell mkdir -p '/content/Real-ESRGAN/frame'
%shell ffmpeg -hwaccel_device 0 -i '/content/Real-ESRGAN/upload/{filename}' '/content/Real-ESRGAN/frame/%08d.png'


ffmpeg version 4.3.2-0york0~18.04 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version='0york0~18.04' --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab

In [ ]:
%shell ffmpeg -hwaccel

# 3. Inference


In [ ]:
#超分
# if it is out of memory, try to use the `--tile` option 爆显存调tile 现值100
!python inference_realesrgan.py --model_path experiments/pretrained_models/RealESRGAN_x4plus.pth --input frame --scale 4 --half --tile 100

流式输出内容被截断，只能显示最后 5000 行内容。
	Tile 16/220
	Tile 17/220
	Tile 18/220
	Tile 19/220
	Tile 20/220
	Tile 21/220
	Tile 22/220
	Tile 23/220
	Tile 24/220
	Tile 25/220
	Tile 26/220
	Tile 27/220
	Tile 28/220
	Tile 29/220
	Tile 30/220
	Tile 31/220
	Tile 32/220
	Tile 33/220
	Tile 34/220
	Tile 35/220
	Tile 36/220
	Tile 37/220
	Tile 38/220
	Tile 39/220
	Tile 40/220
	Tile 41/220
	Tile 42/220
	Tile 43/220
	Tile 44/220
	Tile 45/220
	Tile 46/220
	Tile 47/220
	Tile 48/220
	Tile 49/220
	Tile 50/220
	Tile 51/220
	Tile 52/220
	Tile 53/220
	Tile 54/220
	Tile 55/220
	Tile 56/220
	Tile 57/220
	Tile 58/220
	Tile 59/220
	Tile 60/220
	Tile 61/220
	Tile 62/220
	Tile 63/220
	Tile 64/220
	Tile 65/220
	Tile 66/220
	Tile 67/220
	Tile 68/220
	Tile 69/220
	Tile 70/220
	Tile 71/220
	Tile 72/220
	Tile 73/220
	Tile 74/220
	Tile 75/220
	Tile 76/220
	Tile 77/220
	Tile 78/220
	Tile 79/220
	Tile 80/220
	Tile 81/220
	Tile 82/220
	Tile 83/220
	Tile 84/220
	Tile 85/220
	Tile 86/220
	Tile 87/220
	Tile 88/220
	Tile 89/220
	Tile 90/22

# 5. Download Results


In [ ]:
%shell mkdir -p '/content/Real-ESRGAN/output'

In [ ]:
%shell ffmpeg -i '/content/Real-ESRGAN/results/%08d_out.png' -c:v libx265 -preset:v [ultrafast-placebo] -profile:v main -x265-params "crf=[0-51]:ref=4:rd=3:no-rect=1:no-amp=1:b-intra=1:rdoq-level=2:limit-tu=4:me=3:subme=5:weightb=1:no-strong-intra-smoothing=1:psy-rd=2.0:psy-rdoq=1.0:no-open-gop=1:keyint=200:min-keyint=1:rc-lookahead=20:bframes=6:aq-mode=1:aq-strength=0.8:qg-size=8:cbqpoffs=-2:crqpoffs=-2:qcomp=0.65:deblock=-1:no-sao=1:range=limited:colorprim=1:transfer=1:colormatrix=1:repeat-headers=1" -pix_fmt yuv420p /content/Real-ESRGAN/output/output.mp4

In [ ]:
'/content/Real-ESRGAN/upload/{filename}' '/content/Real-ESRGAN/frame/%08d.png'

In [ ]:
# Download the results
zip_filename = 'Real-ESRGAN_result.zip'
if os.path.exists(zip_filename):
  os.remove(zip_filename)
os.system(f"zip -r -j {zip_filename} results/*")
files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>